In [1]:
from bs4 import BeautifulSoup as BS
import requests
import pandas as pd
import re

In [2]:
df = pd.DataFrame()

URLS_1 = ['https://www.exchange-rates.org/converter/usd-eur', 'https://www.exchange-rates.org/converter/eur-usd', 
         'https://www.exchange-rates.org/converter/uah-usd', 'https://www.exchange-rates.org/converter/usd-uah', 
         'https://www.exchange-rates.org/converter/eur-uah', 'https://www.exchange-rates.org/converter/uah-eur']

for URL_1 in URLS_1:
    
    from_cur = URL_1[-7:].split('-')[0].upper()
    to_cur = URL_1[-7:].split('-')[1].upper()
    
    req_1 = requests.get(URL_1)
    soup = BS(req_1.text, "html.parser")

    orig_path = "https://www.exchange-rates.org"

    urls = [orig_path + i[:-6] for i in re.findall(r'/\S*\d', str(soup.find_all('ul', class_='rates-by-year')[0]))]

    df_for_one = pd.DataFrame()

    for url in urls:
        req_2 = requests.get(url)
        soup_2 = BS(req_2.text, "html.parser")
        dates = [pd.to_datetime(date.text) for date in soup_2.find_all('a', class_='n')]
        values = [float(val.text.split()[-2]) for val in soup_2.find_all('span', class_='n')]
        _ = pd.DataFrame(values, index=dates, columns=[f'1_{from_cur}_to_{to_cur}'])
        df_for_one = pd.concat([df_for_one,_], axis=0)
    
    df_for_one = df_for_one.sort_index()
    display(df_for_one.tail())
        
    df = pd.concat([df, df_for_one], axis=1)

,1_USD_to_EUR
2024-05-30,0.9231
2024-05-31,0.9210
2024-06-03,0.9167
2024-06-04,0.9191
2024-06-05,0.9199


,1_EUR_to_USD
2024-05-30,1.0834
2024-05-31,1.0858
2024-06-03,1.0908
2024-06-04,1.0880
2024-06-05,1.0871


,1_UAH_to_USD
2024-05-30,0.02468
2024-05-31,0.02467
2024-06-03,0.02476
2024-06-04,0.02481
2024-06-05,0.02491


,1_USD_to_UAH
2024-05-30,40.513
2024-05-31,40.539
2024-06-03,40.386
2024-06-04,40.302
2024-06-05,40.139


,1_EUR_to_UAH
2024-05-30,43.890
2024-05-31,44.015
2024-06-03,44.054
2024-06-04,43.848
2024-06-05,43.634


,1_UAH_to_EUR
2024-05-30,0.02278
2024-05-31,0.02272
2024-06-03,0.02270
2024-06-04,0.02281
2024-06-05,0.02292


In [3]:
df = df.dropna().sort_index()
df.index.name = 'date'
df

,1_USD_to_EUR,1_EUR_to_USD,1_UAH_to_USD,1_USD_to_UAH,1_EUR_to_UAH,1_UAH_to_EUR
date,,,,,,
2015-01-01,0.8265,1.2098,0.06322,15.819,19.138,0.05225
2015-01-02,0.8324,1.2014,0.06322,15.819,19.004,0.05262
2015-01-03,0.8335,1.1997,0.06322,15.819,18.978,0.05269
2015-01-04,0.8370,1.1946,0.06265,15.962,18.900,0.05243
2015-01-05,0.8391,1.1918,0.06322,15.818,18.852,0.05305
...,...,...,...,...,...,...
2024-05-30,0.9231,1.0834,0.02468,40.513,43.890,0.02278
2024-05-31,0.9210,1.0858,0.02467,40.539,44.015,0.02272
2024-06-03,0.9167,1.0908,0.02476,40.386,44.054,0.02270


In [4]:
df.to_csv('/diploma_info/datalake/currency_rates.csv')